# QuantAlpha Engine V3 — 缩量稳价因子（VolSpike-PriceStable）完整演示

**本 Notebook 新增因子：**

| 因子 | 核心逻辑 | 信号方向 |
|------|---------|----------|
| `VolSpike` | 换手率近期均值 / 历史均值（量能放大倍数） | 值越大 → 量越放大 |
| `PriceVarShrink` | 价格方差近期 / 历史（波动收缩程度，合并开收盘计算） | 值越小 → 波动越收缩 |
| `PriceMeanShrink` | 均价近期 / 历史（价格趋势下移程度） | 值越小 → 均价越下移 |
| `VolSpikeStablePrice` | 三重信号加权复合（量放大 + 方差收缩 + 均价下移） | 正值越高 → 信号越强 |

**核心捕捉形态：** 成交量突然放大，但价格波动却异常变小的股票
（主力建仓 / 筹码交换的典型量价特征）

---

**演示内容：**

| 章节 | 内容 |
|------|------|
| §1   | 环境准备 + 框架导入 |
| §2   | 模拟数据生成 |
| §3   | `VolSpike` — 换手率放大因子验证与可视化 |
| §4   | `PriceVarShrink` — 价格方差收缩因子验证与可视化 |
| §5   | `PriceMeanShrink` — 均价下移因子验证与可视化 |
| §6   | `VolSpikeStablePrice` — 三重信号复合因子验证 |
| §7   | 参数敏感性分析（n_short / n_long / 阈值扫描）|
| §8   | 单因子回测 + 绩效对比 |
| §9   | 与现有因子库的相关性分析（因子独立性检验）|
| §10  | 多因子融合（StatisticalCombiner）|
| §11  | 总结与参数建议 |

## §1. 环境准备 + 框架导入

In [ ]:
import os, warnings
warnings.filterwarnings('ignore')
print('当前路径:', os.getcwd())
# 如路径不对请取消注释：
# os.chdir(r'D:\Stock\Frame')

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.family'] = 'SimHei'   # Windows 中文字体
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# ── 框架导入 ──────────────────────────────────────────────────
from quant_alpha_engine import (
    MockDataGenerator,
    AlphaOps,
    VectorEngine,
    StatisticalCombiner,
)
from quant_alpha_engine.ops import AlphaOps as op
import quant_alpha_engine as qae

print(f'✅ QuantAlpha Engine v{qae.__version__} 导入成功')

# 确认 V3 新增算子已注册
V3_OPS = ['VolSpike', 'PriceVarShrink', 'PriceMeanShrink', 'VolSpikeStablePrice']
missing = [name for name in V3_OPS if not hasattr(op, name)]
if missing:
    print(f'❌ 以下算子缺失，请检查 alpha_ops.py：{missing}')
else:
    print(f'✅ V3 新增算子（{len(V3_OPS)} 个）全部就绪：', ', '.join(V3_OPS))

## §2. 生成模拟数据

In [ ]:
gen = MockDataGenerator(
    n_stocks     = 100,
    n_days       = 504,   # 约 2 年
    n_industries = 10,
    seed         = 42,
    mu           = 0.08,
    sigma        = 0.30,
)
data = gen.generate()

close    = data.close
open_    = data.open
high     = data.high
low      = data.low
volume   = data.volume
industry = data.industry
is_susp  = data.is_suspended
is_limit = data.is_limit

# ── 构造换手率（模拟：成交量 / 流通股本，这里用成交量归一化代替）──
# 实盘中请直接传入真实换手率（每日成交股数 / 流通股数）
# 此处用 volume / volume.rolling(252).mean() 的比率代理换手率的相对水平
vol_ma252 = volume.rolling(window=252, min_periods=60).mean()
turnover  = volume / vol_ma252.replace(0, np.nan)   # 相对成交量（代理换手率）
turnover  = turnover.clip(lower=0)                   # 确保非负

print(f'价格数据: {close.shape}  |  {close.index[0].date()} ~ {close.index[-1].date()}')
print(f'换手率代理指标: 均值={turnover.stack().dropna().mean():.4f}  '
      f'中位数={turnover.stack().dropna().median():.4f}')
print(f'停牌比例: {is_susp.values.mean()*100:.2f}%  |  涨跌停比例: {is_limit.values.mean()*100:.2f}%')

## §3. `VolSpike` — 换手率放大因子

**核心逻辑：**
```
M1 = mean(turnover, n_short)                         # 近期均换手率
M2 = mean(turnover[-n_short-n_long : -n_short], n_long)  # 历史基准均换手率（不重叠）
VolSpike = M1 / M2
```
**信号：** `VolSpike > vol_mult` → 量能显著放大

In [ ]:
# ── 参数配置 ──────────────────────────────────────────────────
N_SHORT     = 3     # 短期窗口（天）：观察当前异动
N_LONG      = 10    # 长期窗口（天）：历史背景基准
N_MULTIPLE  = 2.0   # 统一的基础倍数参数
VOL_MULT    = 2.0   # VolSpike 的放大阈值：M1 > M2 × vol_mult 为显著放量
PRICEVAR_SHRINK = 0.5   # PriceVarShrink 的收缩阈值：V1 < V2 × pricevar_shrink 为显著收缩
PRICE_SHRINK    = 0.98  # PriceMeanShrink 的下移阈值：P1 < P2 × price_shrink 为显著下移

print(f'参数配置：n_short={N_SHORT}, n_long={N_LONG}, n_multiple={N_MULTIPLE}')
print(f'子因子阈值：vol_mult={VOL_MULT}, pricevar_shrink={PRICEVAR_SHRINK}, price_shrink={PRICE_SHRINK}')

In [ ]:
# ── 计算 VolSpike ────────────────────────────────────────────
f_volspike = op.VolSpike(
    turnover,
    n_short    = N_SHORT,
    n_long     = N_LONG,
    n_multiple = VOL_MULT,
)

vs_vals = f_volspike.stack().dropna()
print('VolSpike(turnover, n_short=3, n_long=10, n_multiple=2.0)')
print(f'  均值: {vs_vals.mean():.4f}  中位数: {vs_vals.median():.4f}')
print(f'  P5 / P95: [{vs_vals.quantile(0.05):.4f}, {vs_vals.quantile(0.95):.4f}]')
print(f'  显著放量（>vol_mult={VOL_MULT}）比例: {(vs_vals > VOL_MULT).mean()*100:.2f}%')
print(f'  NaN 占比: {f_volspike.isna().values.mean()*100:.2f}%（前 {N_SHORT+N_LONG-1} 行应为 NaN）')

In [ ]:
# ── 可视化：VolSpike 截面分布 + 时序走势 ─────────────────────
stock_ex = close.columns[5]
date_slice = slice(-180, None)
idx_vis = close.index[date_slice]

fig, axes = plt.subplots(1, 3, figsize=(16, 4), facecolor='#1A1A2E')

# 左：截面分布（最后一日）
ax0 = axes[0]
ax0.set_facecolor('#16213E')
last_vs = f_volspike.iloc[-1].dropna().values
ax0.hist(last_vs, bins=30, color='#FF6B35', alpha=0.8, edgecolor='none')
ax0.axvline(VOL_MULT, color='#FFD700', linewidth=1.5, linestyle='--', label=f'阈值={VOL_MULT}')
ax0.axvline(np.nanmean(last_vs), color='#ECF0F1', linewidth=1.2, linestyle='--', alpha=0.7)
ax0.set_title('VolSpike 截面分布（最后交易日）', color='#ECF0F1', fontsize=9)
ax0.set_xlabel('M1/M2（换手率放大倍数）', color='#ECF0F1', fontsize=8)
ax0.legend(fontsize=7, facecolor='#1A1A2E', edgecolor='none', labelcolor='#ECF0F1')
ax0.tick_params(colors='#ECF0F1', labelsize=7)
for sp in ax0.spines.values(): sp.set_color('#0F3460')

# 中：单只股票时序走势
ax1 = axes[1]
ax1.set_facecolor('#16213E')
vs_ts = f_volspike.loc[idx_vis, stock_ex]
ax1.plot(idx_vis, vs_ts.values, color='#FF6B35', linewidth=1.2, alpha=0.9)
ax1.axhline(VOL_MULT, color='#FFD700', linewidth=1.2, linestyle='--', alpha=0.8, label=f'阈值={VOL_MULT}')
ax1.axhline(1.0, color='#ECF0F1', linewidth=0.6, linestyle=':', alpha=0.5, label='基准=1.0')
ax1.fill_between(idx_vis, vs_ts.values, VOL_MULT,
                  where=(vs_ts.values > VOL_MULT),
                  color='#FF6B35', alpha=0.3, label='显著放量区间')
ax1.set_title(f'VolSpike 时序 — {stock_ex}（近180日）', color='#ECF0F1', fontsize=9)
ax1.set_ylabel('换手率放大比（M1/M2）', color='#ECF0F1', fontsize=8)
ax1.legend(fontsize=7, facecolor='#1A1A2E', edgecolor='none', labelcolor='#ECF0F1')
ax1.tick_params(colors='#ECF0F1', labelsize=7)
for sp in ax1.spines.values(): sp.set_color('#0F3460')

# 右：对应股票价格走势（对照）
ax2 = axes[2]
ax2.set_facecolor('#16213E')
price_ts = close.loc[idx_vis, stock_ex]
ax2.plot(idx_vis, price_ts.values, color='#2ECC71', linewidth=1.2, alpha=0.9)
# 标记放量点
spike_mask = vs_ts.values > VOL_MULT
spike_idx = [idx_vis[i] for i in range(len(idx_vis)) if spike_mask[i] and not np.isnan(spike_mask[i])]
spike_prices = [price_ts.iloc[i] for i in range(len(idx_vis)) if spike_mask[i] and not np.isnan(spike_mask[i])]
ax2.scatter(spike_idx, spike_prices, color='#FF6B35', s=20, zorder=5, label='放量时刻')
ax2.set_title(f'对应股价 — {stock_ex}（近180日）', color='#ECF0F1', fontsize=9)
ax2.set_ylabel('价格', color='#ECF0F1', fontsize=8)
ax2.legend(fontsize=7, facecolor='#1A1A2E', edgecolor='none', labelcolor='#ECF0F1')
ax2.tick_params(colors='#ECF0F1', labelsize=7)
for sp in ax2.spines.values(): sp.set_color('#0F3460')

fig.suptitle('VolSpike 换手率放大因子分析', color='#ECF0F1', fontsize=11, y=1.02)
plt.tight_layout()
plt.show()
print('✅ VolSpike 验证完成')

## §4. `PriceVarShrink` — 价格方差收缩因子

**核心逻辑：**
```
combined_short = concat(close[-n_short:], open_[-n_short:])  # 2×n_short 个价格点
combined_long  = concat(close[-n_short-n_long:-n_short], open_[...])  # 2×n_long 个价格点
V1 = var(combined_short)
V2 = var(combined_long)
PriceVarShrink = V1 / V2
```
**信号：** `PriceVarShrink < pricevar_shrink` → 价格波动显著收缩

In [ ]:
# ── 计算 PriceVarShrink ───────────────────────────────────────
f_pvshrink = op.PriceVarShrink(
    close,
    open_,
    n_short    = N_SHORT,
    n_long     = N_LONG,
    n_multiple = PRICEVAR_SHRINK,
)

pvs_vals = f_pvshrink.stack().dropna()
print('PriceVarShrink(close, open_, n_short=3, n_long=10, n_multiple=0.5)')
print(f'  均值: {pvs_vals.mean():.4f}  中位数: {pvs_vals.median():.4f}')
print(f'  P5 / P95: [{pvs_vals.quantile(0.05):.4f}, {pvs_vals.quantile(0.95):.4f}]')
print(f'  显著收缩（<pricevar_shrink={PRICEVAR_SHRINK}）比例: {(pvs_vals < PRICEVAR_SHRINK).mean()*100:.2f}%')
print(f'  >1（近期波动更大）比例: {(pvs_vals > 1.0).mean()*100:.2f}%')
print(f'  <1（近期波动更小）比例: {(pvs_vals < 1.0).mean()*100:.2f}%')

In [ ]:
# ── 可视化：PriceVarShrink 分析 ──────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(16, 4), facecolor='#1A1A2E')

# 左：截面分布（最后一日）
ax0 = axes[0]
ax0.set_facecolor('#16213E')
last_pvs = f_pvshrink.iloc[-1].dropna().values
last_pvs_clipped = np.clip(last_pvs, 0, 5)  # 裁剪极端值以便可视化
ax0.hist(last_pvs_clipped, bins=30, color='#9B59B6', alpha=0.8, edgecolor='none')
ax0.axvline(PRICEVAR_SHRINK, color='#FFD700', linewidth=1.5, linestyle='--',
             label=f'阈值={PRICEVAR_SHRINK}')
ax0.axvline(1.0, color='#ECF0F1', linewidth=1.0, linestyle=':', alpha=0.6, label='基准=1.0')
ax0.set_title('PriceVarShrink 截面分布（最后交易日）', color='#ECF0F1', fontsize=9)
ax0.set_xlabel('V1/V2（方差收缩比，<1 表示收缩）', color='#ECF0F1', fontsize=7)
ax0.legend(fontsize=7, facecolor='#1A1A2E', edgecolor='none', labelcolor='#ECF0F1')
ax0.tick_params(colors='#ECF0F1', labelsize=7)
for sp in ax0.spines.values(): sp.set_color('#0F3460')

# 中：单只股票时序
ax1 = axes[1]
ax1.set_facecolor('#16213E')
pvs_ts = f_pvshrink.loc[idx_vis, stock_ex].clip(upper=4)
ax1.plot(idx_vis, pvs_ts.values, color='#9B59B6', linewidth=1.2)
ax1.axhline(PRICEVAR_SHRINK, color='#FFD700', linewidth=1.2, linestyle='--',
             alpha=0.8, label=f'阈值={PRICEVAR_SHRINK}')
ax1.axhline(1.0, color='#ECF0F1', linewidth=0.6, linestyle=':', alpha=0.5, label='基准=1.0')
ax1.fill_between(idx_vis, pvs_ts.values, PRICEVAR_SHRINK,
                  where=(pvs_ts.values < PRICEVAR_SHRINK),
                  color='#9B59B6', alpha=0.3, label='显著收缩区间')
ax1.set_title(f'PriceVarShrink 时序 — {stock_ex}（近180日）', color='#ECF0F1', fontsize=9)
ax1.set_ylabel('V1/V2（方差收缩比）', color='#ECF0F1', fontsize=8)
ax1.legend(fontsize=7, facecolor='#1A1A2E', edgecolor='none', labelcolor='#ECF0F1')
ax1.tick_params(colors='#ECF0F1', labelsize=7)
for sp in ax1.spines.values(): sp.set_color('#0F3460')

# 右：VolSpike vs PriceVarShrink 散点（验证联合信号稀有性）
ax2 = axes[2]
ax2.set_facecolor('#16213E')
xy = pd.concat([
    f_volspike.iloc[-60:].stack().rename('vs'),
    f_pvshrink.iloc[-60:].stack().rename('pvs'),
], axis=1).dropna()
xy_clip = xy.clip(upper={'vs': 6, 'pvs': 4})
# 颜色编码：同时满足 vs>vol_mult 且 pvs<pricevar_shrink 的点高亮
joint_signal = (xy['vs'] > VOL_MULT) & (xy['pvs'] < PRICEVAR_SHRINK)
ax2.scatter(xy_clip.loc[~joint_signal, 'vs'], xy_clip.loc[~joint_signal, 'pvs'],
             c='#3498DB', alpha=0.2, s=6, label='普通')
ax2.scatter(xy_clip.loc[joint_signal, 'vs'], xy_clip.loc[joint_signal, 'pvs'],
             c='#FF6B35', alpha=0.8, s=20, label=f'联合信号 ({joint_signal.sum()}只)')
ax2.axvline(VOL_MULT, color='#FFD700', linewidth=0.8, linestyle='--', alpha=0.6)
ax2.axhline(PRICEVAR_SHRINK, color='#FFD700', linewidth=0.8, linestyle='--', alpha=0.6)
ax2.set_title('VolSpike vs PriceVarShrink\n（近60日截面，橙点=联合信号）', color='#ECF0F1', fontsize=9)
ax2.set_xlabel('VolSpike（量能放大倍数）', color='#ECF0F1', fontsize=7)
ax2.set_ylabel('PriceVarShrink（方差收缩比）', color='#ECF0F1', fontsize=7)
ax2.legend(fontsize=7, facecolor='#1A1A2E', edgecolor='none', labelcolor='#ECF0F1')
ax2.tick_params(colors='#ECF0F1', labelsize=7)
for sp in ax2.spines.values(): sp.set_color('#0F3460')

fig.suptitle('PriceVarShrink 价格方差收缩因子分析', color='#ECF0F1', fontsize=11, y=1.02)
plt.tight_layout()
plt.show()
print('✅ PriceVarShrink 验证完成')

## §5. `PriceMeanShrink` — 均价下移因子

**核心逻辑：**
```
P1 = mean(close[-n_short:])                    # 近期均价
P2 = mean(close[-n_short-n_long:-n_short])     # 基准期均价（不重叠）
PriceMeanShrink = P1 / P2
```
**信号：** `PriceMeanShrink < price_shrink` → 均价显著下移（<1 表示价格下行）

In [ ]:
# ── 计算 PriceMeanShrink ─────────────────────────────────────
f_pmshrink = op.PriceMeanShrink(
    close,
    n_short    = N_SHORT,
    n_long     = N_LONG,
    n_multiple = PRICE_SHRINK,
)

pms_vals = f_pmshrink.stack().dropna()
print('PriceMeanShrink(close, n_short=3, n_long=10, n_multiple=0.98)')
print(f'  均值: {pms_vals.mean():.6f}  中位数: {pms_vals.median():.6f}')
print(f'  P1 / P99: [{pms_vals.quantile(0.01):.4f}, {pms_vals.quantile(0.99):.4f}]')
print(f'  显著下移（<price_shrink={PRICE_SHRINK}）比例: {(pms_vals < PRICE_SHRINK).mean()*100:.2f}%')
print(f'  均价上移（>1.0）比例: {(pms_vals > 1.0).mean()*100:.2f}%')
print(f'  均价下移（<1.0）比例: {(pms_vals < 1.0).mean()*100:.2f}%')
print(f'  均值接近 1.0: {abs(pms_vals.mean() - 1.0) < 0.01}（合理，近远期均价应大致相近）')

In [ ]:
# ── 可视化：PriceMeanShrink 分析 ─────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(16, 4), facecolor='#1A1A2E')

# 左：截面分布
ax0 = axes[0]
ax0.set_facecolor('#16213E')
last_pms = f_pmshrink.iloc[-1].dropna().values
ax0.hist(last_pms, bins=30, color='#1ABC9C', alpha=0.8, edgecolor='none')
ax0.axvline(PRICE_SHRINK, color='#FFD700', linewidth=1.5, linestyle='--',
             label=f'阈值={PRICE_SHRINK}')
ax0.axvline(1.0, color='#ECF0F1', linewidth=1.0, linestyle=':', alpha=0.6, label='基准=1.0')
ax0.set_title('PriceMeanShrink 截面分布（最后交易日）', color='#ECF0F1', fontsize=9)
ax0.set_xlabel('P1/P2（均价比，<1 表示下移）', color='#ECF0F1', fontsize=8)
ax0.legend(fontsize=7, facecolor='#1A1A2E', edgecolor='none', labelcolor='#ECF0F1')
ax0.tick_params(colors='#ECF0F1', labelsize=7)
for sp in ax0.spines.values(): sp.set_color('#0F3460')

# 中：单只股票时序
ax1 = axes[1]
ax1.set_facecolor('#16213E')
pms_ts = f_pmshrink.loc[idx_vis, stock_ex]
colors_fill = ['#E74C3C' if v < 1.0 else '#2ECC71' for v in pms_ts.fillna(1.0).values]
ax1.bar(idx_vis, (pms_ts.values - 1.0) * 100, color=colors_fill, alpha=0.7, width=0.8)
ax1.axhline(0, color='#ECF0F1', linewidth=0.8, alpha=0.6)
ax1.axhline((PRICE_SHRINK - 1.0) * 100, color='#FFD700', linewidth=1.2,
             linestyle='--', alpha=0.8, label=f'阈值={(PRICE_SHRINK-1)*100:.0f}%')
ax1.set_title(f'PriceMeanShrink 时序 — {stock_ex}（近180日）', color='#ECF0F1', fontsize=9)
ax1.set_ylabel('均价变化率（%）', color='#ECF0F1', fontsize=8)
ax1.legend(fontsize=7, facecolor='#1A1A2E', edgecolor='none', labelcolor='#ECF0F1')
ax1.tick_params(colors='#ECF0F1', labelsize=7)
for sp in ax1.spines.values(): sp.set_color('#0F3460')

# 右：三因子联合信号分析（同时满足三条件的频率）
ax2 = axes[2]
ax2.set_facecolor('#16213E')
# 计算每日截面中三信号同时满足的股票数量
cond1 = f_volspike > VOL_MULT
cond2 = f_pvshrink < PRICEVAR_SHRINK
cond3 = f_pmshrink < PRICE_SHRINK
triple_count = (cond1 & cond2 & cond3).sum(axis=1)
ax2.fill_between(triple_count.index, triple_count.values,
                  color='#FF6B35', alpha=0.7)
ax2.plot(triple_count.index, triple_count.values, color='#FFD700', linewidth=0.8)
ax2.set_title('每日三重信号同时满足的股票数量', color='#ECF0F1', fontsize=9)
ax2.set_ylabel('股票数量', color='#ECF0F1', fontsize=8)
ax2.set_xlabel('日期', color='#ECF0F1', fontsize=8)
ax2.tick_params(colors='#ECF0F1', labelsize=7)
for sp in ax2.spines.values(): sp.set_color('#0F3460')
mean_cnt = triple_count.mean()
ax2.axhline(mean_cnt, color='#ECF0F1', linewidth=0.8, linestyle='--',
             alpha=0.6, label=f'均值={mean_cnt:.1f}只')
ax2.legend(fontsize=7, facecolor='#1A1A2E', edgecolor='none', labelcolor='#ECF0F1')

fig.suptitle('PriceMeanShrink 均价下移因子分析', color='#ECF0F1', fontsize=11, y=1.02)
plt.tight_layout()
plt.show()
print('✅ PriceMeanShrink 验证完成')

## §6. `VolSpikeStablePrice` — 三重信号复合因子

**合成逻辑：**
```
Z1 = ZScore(VolSpike)            # 量能放大 → 正向
Z2 = ZScore(-PriceVarShrink)     # 方差收缩 → 取负后正向
Z3 = ZScore(-PriceMeanShrink)    # 均价下移 → 取负后正向
VolSpikeStablePrice = (Z1 + Z2 + Z3) / 3
```

In [ ]:
# ── 计算复合因子 VolSpikeStablePrice ─────────────────────────
f_vssp = op.VolSpikeStablePrice(
    turnover,
    close,
    open_,
    n_short         = N_SHORT,
    n_long          = N_LONG,
    n_multiple      = N_MULTIPLE,
    vol_mult        = VOL_MULT,
    pricevar_shrink = PRICEVAR_SHRINK,
    price_shrink    = PRICE_SHRINK,
)

vssp_vals = f_vssp.stack().dropna()
print('VolSpikeStablePrice 复合因子')
print(f'  形状: {f_vssp.shape}')
print(f'  均值: {vssp_vals.mean():.6f}  （截面 ZScore 理论均值接近 0）')
print(f'  标准差: {vssp_vals.std():.4f}')
print(f'  P1 / P99: [{vssp_vals.quantile(0.01):.4f}, {vssp_vals.quantile(0.99):.4f}]')
print(f'  NaN 占比: {f_vssp.isna().values.mean()*100:.2f}%')

# 验证子因子与复合因子的相关性
print('\n子因子与复合因子的相关性（截面平均 Pearson 相关）:')
for name, sub_f in [('VolSpike', f_volspike),
                     ('PriceVarShrink', f_pvshrink),
                     ('PriceMeanShrink', f_pmshrink)]:
    # 截面相关（每日计算 Pearson，取均值）
    daily_corr = []
    for date in f_vssp.index:
        row_comp = f_vssp.loc[date].dropna()
        row_sub  = sub_f.loc[date].dropna()
        common   = row_comp.index.intersection(row_sub.index)
        if len(common) > 5:
            c = row_comp[common].corr(row_sub[common])
            if not np.isnan(c):
                daily_corr.append(c)
    print(f'  {name:<20}: {np.mean(daily_corr):+.4f}')

In [ ]:
# ── 可视化：复合因子综合分析 ─────────────────────────────────
fig = plt.figure(figsize=(16, 8), facecolor='#1A1A2E')
gs  = gridspec.GridSpec(2, 3, figure=fig, hspace=0.4, wspace=0.3)

# 上左：截面分布（最后一日）
ax0 = fig.add_subplot(gs[0, 0])
ax0.set_facecolor('#16213E')
last_vssp = f_vssp.iloc[-1].dropna().values
ax0.hist(last_vssp, bins=25, color='#FF6B35', alpha=0.8, edgecolor='none')
ax0.axvline(0, color='#ECF0F1', linewidth=1.0, linestyle='--', alpha=0.5)
ax0.set_title('复合因子截面分布', color='#ECF0F1', fontsize=9)
ax0.tick_params(colors='#ECF0F1', labelsize=7)
for sp in ax0.spines.values(): sp.set_color('#0F3460')

# 上中：Top10 vs Bottom10 股票的子因子对比
ax1 = fig.add_subplot(gs[0, 1])
ax1.set_facecolor('#16213E')
last_vssp_series = f_vssp.iloc[-1].dropna().sort_values(ascending=False)
top10  = last_vssp_series.head(10).index
bot10  = last_vssp_series.tail(10).index
sub_data = {
    'VolSpike':        [f_volspike.iloc[-1][top10].mean(),  f_volspike.iloc[-1][bot10].mean()],
    'PriceVarShrink':  [f_pvshrink.iloc[-1][top10].mean(),  f_pvshrink.iloc[-1][bot10].mean()],
    'PriceMeanShrink': [f_pmshrink.iloc[-1][top10].mean(),  f_pmshrink.iloc[-1][bot10].mean()],
}
x = np.arange(3)
w = 0.3
ax1.bar(x - w/2, [sub_data[k][0] for k in sub_data], w,
         label='Top10（信号强）', color='#FF6B35', alpha=0.8)
ax1.bar(x + w/2, [sub_data[k][1] for k in sub_data], w,
         label='Bot10（信号弱）', color='#3498DB', alpha=0.8)
ax1.set_xticks(x)
ax1.set_xticklabels(['VolSpike', 'VarShrink', 'MeanShrink'], color='#ECF0F1', fontsize=7)
ax1.set_title('Top/Bot10 子因子均值对比', color='#ECF0F1', fontsize=9)
ax1.legend(fontsize=7, facecolor='#1A1A2E', edgecolor='none', labelcolor='#ECF0F1')
ax1.tick_params(colors='#ECF0F1', labelsize=7)
for sp in ax1.spines.values(): sp.set_color('#0F3460')

# 上右：IC 时序（截面 Rank IC）
ax2 = fig.add_subplot(gs[0, 2])
ax2.set_facecolor('#16213E')
fwd_ret = close.pct_change(5).shift(-5)  # 5日前向收益率
rank_ic_series = []
for date in f_vssp.index[:-5]:
    factor_cs = f_vssp.loc[date].dropna()
    ret_cs    = fwd_ret.loc[date].dropna()
    common    = factor_cs.index.intersection(ret_cs.index)
    if len(common) > 10:
        ic_val = factor_cs[common].rank().corr(ret_cs[common].rank())
        rank_ic_series.append({'date': date, 'ic': ic_val})
ic_df = pd.DataFrame(rank_ic_series).set_index('date')['ic']
ic_rolling = ic_df.rolling(20).mean()
ax2.plot(ic_df.index, ic_df.values, color='#3498DB', linewidth=0.6, alpha=0.5, label='日 IC')
ax2.plot(ic_rolling.index, ic_rolling.values, color='#FF6B35', linewidth=1.5, label='20日均 IC')
ax2.axhline(0, color='#ECF0F1', linewidth=0.6, alpha=0.4, linestyle='--')
ic_mean = ic_df.mean()
ic_std  = ic_df.std()
ax2.set_title(f'5日前向 IC 时序\nIC均值={ic_mean:.4f}  ICIR={ic_mean/ic_std:.4f}',
               color='#ECF0F1', fontsize=9)
ax2.legend(fontsize=7, facecolor='#1A1A2E', edgecolor='none', labelcolor='#ECF0F1')
ax2.tick_params(colors='#ECF0F1', labelsize=7)
for sp in ax2.spines.values(): sp.set_color('#0F3460')

# 下左：子因子的截面相关矩阵（热力图）
ax3 = fig.add_subplot(gs[1, 0])
ax3.set_facecolor('#16213E')
# 计算截面平均相关矩阵
all_sub = {'VolSpike': f_volspike, 'VarShrink': f_pvshrink, 'MeanShrink': f_pmshrink, 'Composite': f_vssp}
corr_vals = pd.DataFrame(index=all_sub.keys(), columns=all_sub.keys(), dtype=float)
for k1, f1 in all_sub.items():
    for k2, f2 in all_sub.items():
        daily_c = []
        for d in f1.index[-60:]:
            s1 = f1.loc[d].dropna()
            s2 = f2.loc[d].dropna()
            cm = s1.index.intersection(s2.index)
            if len(cm) > 5:
                c = s1[cm].corr(s2[cm])
                if not np.isnan(c): daily_c.append(c)
        corr_vals.loc[k1, k2] = np.mean(daily_c) if daily_c else np.nan
corr_arr = corr_vals.astype(float).values
im = ax3.imshow(corr_arr, cmap='RdYlGn', vmin=-1, vmax=1, aspect='auto')
ax3.set_xticks(range(4))
ax3.set_yticks(range(4))
ax3.set_xticklabels(list(all_sub.keys()), color='#ECF0F1', fontsize=7, rotation=30)
ax3.set_yticklabels(list(all_sub.keys()), color='#ECF0F1', fontsize=7)
for i in range(4):
    for j in range(4):
        ax3.text(j, i, f'{corr_arr[i,j]:.2f}', ha='center', va='center',
                  color='black', fontsize=8)
ax3.set_title('子因子 & 复合因子相关矩阵（近60日）', color='#ECF0F1', fontsize=9)
plt.colorbar(im, ax=ax3, fraction=0.046)

# 下中：IC 累积图
ax4 = fig.add_subplot(gs[1, 1])
ax4.set_facecolor('#16213E')
ax4.plot(ic_df.index, ic_df.cumsum().values, color='#2ECC71', linewidth=1.5)
ax4.axhline(0, color='#ECF0F1', linewidth=0.5, alpha=0.4, linestyle='--')
ax4.set_title('IC 累积序列（斜率 = ICIR 方向）', color='#ECF0F1', fontsize=9)
ax4.set_ylabel('累积 IC', color='#ECF0F1', fontsize=8)
ax4.tick_params(colors='#ECF0F1', labelsize=7)
for sp in ax4.spines.values(): sp.set_color('#0F3460')

# 下右：IC 分布直方图
ax5 = fig.add_subplot(gs[1, 2])
ax5.set_facecolor('#16213E')
ax5.hist(ic_df.dropna().values, bins=30, color='#3498DB', alpha=0.8, edgecolor='none')
ax5.axvline(0, color='#ECF0F1', linewidth=0.8, linestyle='--', alpha=0.5)
ax5.axvline(ic_mean, color='#FF6B35', linewidth=1.5, linestyle='--', label=f'均值={ic_mean:.4f}')
ic_win_rate = (ic_df > 0).mean()
ax5.set_title(f'IC 分布  胜率={ic_win_rate*100:.1f}%\n（正值比例，>50% 表示因子有效）',
               color='#ECF0F1', fontsize=9)
ax5.legend(fontsize=7, facecolor='#1A1A2E', edgecolor='none', labelcolor='#ECF0F1')
ax5.tick_params(colors='#ECF0F1', labelsize=7)
for sp in ax5.spines.values(): sp.set_color('#0F3460')

fig.suptitle('VolSpikeStablePrice 复合因子综合分析', color='#ECF0F1', fontsize=12, y=1.01)
plt.show()
print('✅ VolSpikeStablePrice 复合因子验证完成')

## §7. 参数敏感性分析

扫描 `n_short`、`n_long`、`vol_mult`、`pricevar_shrink` 对因子 IC 的影响

In [ ]:
# ── 工具函数：计算因子均值 IC ─────────────────────────────────
def calc_ic(factor: pd.DataFrame, fwd_ret: pd.DataFrame, horizon: int = 5) -> dict:
    """计算因子的 IC_mean、IC_std、ICIR、IC 胜率。"""
    fwd = fwd_ret.pct_change(horizon).shift(-horizon)
    daily_ic = []
    for date in factor.index[:-horizon]:
        f_cs  = factor.loc[date].dropna()
        r_cs  = fwd.loc[date].dropna()
        common = f_cs.index.intersection(r_cs.index)
        if len(common) > 10:
            ic = f_cs[common].rank().corr(r_cs[common].rank())
            if not np.isnan(ic):
                daily_ic.append(ic)
    if not daily_ic:
        return {'IC_Mean': np.nan, 'ICIR': np.nan, 'IC_Win': np.nan}
    ic_arr = np.array(daily_ic)
    ic_std = ic_arr.std() + 1e-10
    return {
        'IC_Mean': ic_arr.mean(),
        'ICIR':    ic_arr.mean() / ic_std,
        'IC_Win':  (ic_arr > 0).mean(),
    }

print('参数扫描中，请稍候...')

# ── 扫描 n_short × n_long ──────────────────────────────────────
n_short_list = [2, 3, 5, 7]
n_long_list  = [5, 10, 15, 20]

scan_results = []
for ns in n_short_list:
    for nl in n_long_list:
        if ns >= nl:
            continue
        f_tmp = op.VolSpikeStablePrice(
            turnover, close, open_,
            n_short=ns, n_long=nl,
            vol_mult=VOL_MULT, pricevar_shrink=PRICEVAR_SHRINK, price_shrink=PRICE_SHRINK,
        )
        metrics = calc_ic(f_tmp, close, horizon=5)
        scan_results.append({'n_short': ns, 'n_long': nl, **metrics})

scan_df = pd.DataFrame(scan_results)
scan_df_fmt = scan_df.copy()
scan_df_fmt['IC_Mean'] = scan_df['IC_Mean'].apply(lambda x: f'{x:+.4f}')
scan_df_fmt['ICIR']    = scan_df['ICIR'].apply(lambda x: f'{x:+.4f}')
scan_df_fmt['IC_Win']  = scan_df['IC_Win'].apply(lambda x: f'{x*100:.1f}%')
print('\n── n_short × n_long 参数扫描（VolSpikeStablePrice）──')
print(scan_df_fmt.to_string(index=False))

In [ ]:
# ── 可视化：IC_Mean 热力图 ────────────────────────────────────
pivot_ic   = scan_df.pivot(index='n_short', columns='n_long', values='IC_Mean')
pivot_icir = scan_df.pivot(index='n_short', columns='n_long', values='ICIR')

fig, axes = plt.subplots(1, 2, figsize=(12, 4), facecolor='#1A1A2E')

for ax, data, title in zip(
    axes,
    [pivot_ic, pivot_icir],
    ['IC 均值热力图', 'ICIR 热力图'],
):
    ax.set_facecolor('#16213E')
    arr = data.values.astype(float)
    vmax = np.nanmax(np.abs(arr))
    im = ax.imshow(arr, cmap='RdYlGn', vmin=-vmax, vmax=vmax, aspect='auto')
    ax.set_xticks(range(len(data.columns)))
    ax.set_yticks(range(len(data.index)))
    ax.set_xticklabels([f'n_long={c}' for c in data.columns], color='#ECF0F1', fontsize=8)
    ax.set_yticklabels([f'n_short={r}' for r in data.index], color='#ECF0F1', fontsize=8)
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            if not np.isnan(arr[i, j]):
                ax.text(j, i, f'{arr[i,j]:.3f}', ha='center', va='center',
                         color='black', fontsize=9, fontweight='bold')
    ax.set_title(f'VolSpikeStablePrice {title}\n（5日前向 Rank IC）',
                  color='#ECF0F1', fontsize=10)
    plt.colorbar(im, ax=ax, fraction=0.046)

plt.tight_layout()
plt.show()
print('✅ 参数敏感性分析完成')

## §8. 单因子回测 + 绩效对比

对四个新因子分别做完整回测，对比各自的 Sharpe / IC / 最大回撤

In [ ]:
# ── 回测公共配置 ──────────────────────────────────────────────
REBALANCE_FREQ = 5        # 调仓频率：5 = 每周
TOP_N          = 30       # 持仓股数
WEIGHT_METHOD  = 'equal'  # 'equal' | 'factor_weighted'
COST_RATE      = 0.0015   # 单边交易成本（0.15%）
DELAY          = 1        # T-1 日因子预测 T 日收益（严格防未来函数）
DECAY          = 0        # 不做线性衰减
INDUSTRY       = None     # 不做行业中性化

# ── 回测时间范围控制（v2.1）──────────────────────────────────────
# None = 使用全部数据；填入日期字符串 = 仅评估指定区间
# 注意：仅裁剪回测评估区间，不影响因子历史窗口的计算
START_DATE = None  # 回测开始日期，如 '2022-06-01'
END_DATE   = None  # 回测结束日期，如 '2023-12-31'

print(f'回测配置：freq={REBALANCE_FREQ}d, top_n={TOP_N}, cost={COST_RATE:.4%}, delay={DELAY}')
print(f'时间范围：{START_DATE} ~ {END_DATE}  (None = 不限制)')

In [ ]:
# ── 因子定义（对齐 n_short / n_long 配置）───────────────────
factors_to_test = {
    'VolSpike':             op.Rank(f_volspike),
    'PriceVarShrink_inv':   op.Rank(-f_pvshrink),   # 收缩越大 → 排名越高
    'PriceMeanShrink_inv':  op.Rank(-f_pmshrink),   # 均价下移 → 排名越高
    'VolSpikeStablePrice':  op.Rank(f_vssp),
}

# 同期参考因子（来自 V1/V2 因子库）
factors_to_test['Reversal_5d']  = op.Rank(-op.Ts_Delta(close, 5))
factors_to_test['LowVol_20d']   = op.Rank(-op.Ts_Std(close.pct_change(), 20))

print('待回测因子：', list(factors_to_test.keys()))

In [ ]:
# ── 批量回测 ──────────────────────────────────────────────────
backtest_results = {}
for name, factor in factors_to_test.items():
    res = VectorEngine(
        factor         = factor,
        close          = close,
        is_suspended   = is_susp,
        is_limit       = is_limit,
        rebalance_freq = REBALANCE_FREQ,
        top_n          = TOP_N,
        weight_method  = WEIGHT_METHOD,
        cost_rate      = COST_RATE,
        delay          = DELAY,
        decay          = DECAY,
        industry       = INDUSTRY,
        start_date     = START_DATE,
        end_date       = END_DATE,
    ).run()
    backtest_results[name] = res
    print(f'  ✅ {name:<28} Sharpe={res.metrics["Sharpe_Ratio"]:+.4f}  '
          f'IC={res.metrics["IC_Mean"]:+.4f}  MaxDD={res.metrics["最大回撤"]*100:+.2f}%')

In [ ]:
# ── 绩效对比表 ────────────────────────────────────────────────
key_cols = ['年化收益率', 'Sharpe_Ratio', '最大回撤', 'IC_Mean', 'ICIR', 'IC_胜率', 'Fitness']
rows = {name: {k: res.metrics.get(k, np.nan) for k in key_cols}
        for name, res in backtest_results.items()}
df_cmp = pd.DataFrame(rows).T

# 格式化
df_fmt = df_cmp.copy()
for c in ['年化收益率', '最大回撤', 'IC_胜率']:
    df_fmt[c] = df_cmp[c].apply(lambda v: f'{v*100:+.2f}%' if pd.notna(v) else 'N/A')
for c in ['Sharpe_Ratio', 'IC_Mean', 'ICIR', 'Fitness']:
    df_fmt[c] = df_cmp[c].apply(lambda v: f'{v:+.4f}' if pd.notna(v) else 'N/A')

print('\n======= V3 新因子 vs 参考因子 — 回测绩效对比 =======')
print(df_fmt.to_string())
print()

# 高亮最优
best_sharpe = df_cmp['Sharpe_Ratio'].idxmax()
best_ic     = df_cmp['IC_Mean'].idxmax()
print(f'🏆 最高 Sharpe：{best_sharpe}  ({df_cmp.loc[best_sharpe, "Sharpe_Ratio"]:+.4f})')
print(f'🏆 最高 IC均值：{best_ic}  ({df_cmp.loc[best_ic, "IC_Mean"]:+.4f})')

In [ ]:
# ── 净值曲线对比 ─────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(14, 6), facecolor='#1A1A2E')
ax.set_facecolor('#16213E')

# 颜色方案
color_map = {
    'VolSpike':             '#FF6B35',
    'PriceVarShrink_inv':   '#9B59B6',
    'PriceMeanShrink_inv':  '#1ABC9C',
    'VolSpikeStablePrice':  '#FFD700',
    'Reversal_5d':          '#95A5A6',
    'LowVol_20d':           '#7F8C8D',
}
lw_map = {
    'VolSpikeStablePrice': 2.5,
}
ls_map = {
    'Reversal_5d': '--',
    'LowVol_20d':  ':',
}

for name, res in backtest_results.items():
    nav = res.nav.dropna()
    color = color_map.get(name, '#ECF0F1')
    lw    = lw_map.get(name, 1.2)
    ls    = ls_map.get(name, '-')
    ax.plot(nav.index, nav.values, label=name, color=color, linewidth=lw, linestyle=ls, alpha=0.9)

ax.axhline(1.0, color='white', linewidth=0.5, alpha=0.3, linestyle='--')
ax.set_title('净值曲线对比：V3 新因子 vs 参考因子', color='#ECF0F1', fontsize=12, pad=10)
ax.set_xlabel('日期', color='#ECF0F1', fontsize=9)
ax.set_ylabel('净值', color='#ECF0F1', fontsize=9)
ax.legend(fontsize=8, facecolor='#1A1A2E', edgecolor='#0F3460', labelcolor='#ECF0F1')
ax.tick_params(colors='#ECF0F1', labelsize=7)
ax.grid(True, color='#2F3640', alpha=0.5, linewidth=0.5)
for sp in ax.spines.values(): sp.set_color('#0F3460')

plt.tight_layout()
plt.show()
print('✅ 回测对比完成')

## §9. 与现有因子库的相关性分析（因子独立性检验）

验证 V3 新因子与 V1/V2 因子库之间的信息独立性（相关性越低 → 融合价值越高）

In [ ]:
# ── 构造代表性因子池 ──────────────────────────────────────────
factor_pool = {
    # V3 新因子
    'VolSpike':            f_volspike,
    'PriceVarShrink':      f_pvshrink,
    'PriceMeanShrink':     f_pmshrink,
    'VSSP (composite)':    f_vssp,
    # V1/V2 参考因子
    'Reversal_5d':         -op.Ts_Delta(close, 5),
    'LowVol_20d':          -op.Ts_Std(close.pct_change(), 20),
    'Momentum_20d':        op.Ts_Delta(close, 20),
    'VWAP_Bias_10d':       op.VWAP_Bias(close, volume, 10),
    'Amihud_20d':          op.Amihud(close, volume, 20),
    'RSI_14d':             op.RSI(close, 14),
}

print(f'因子池规模：{len(factor_pool)} 个因子')

# ── 计算截面平均相关矩阵 ──────────────────────────────────────
names = list(factor_pool.keys())
n = len(names)
avg_corr = np.full((n, n), np.nan)

# 取近 60 个交易日的截面平均相关
dates_sample = close.index[-60:]
for i, k1 in enumerate(names):
    for j, k2 in enumerate(names):
        if i > j:
            avg_corr[i, j] = avg_corr[j, i]
            continue
        if i == j:
            avg_corr[i, j] = 1.0
            continue
        daily_c = []
        for d in dates_sample:
            s1 = factor_pool[k1].loc[d].dropna()
            s2 = factor_pool[k2].loc[d].dropna()
            cm = s1.index.intersection(s2.index)
            if len(cm) > 5:
                c = s1[cm].corr(s2[cm])
                if not np.isnan(c): daily_c.append(c)
        avg_corr[i, j] = avg_corr[j, i] = np.mean(daily_c) if daily_c else np.nan

corr_df = pd.DataFrame(avg_corr, index=names, columns=names)
print('\n截面平均相关矩阵（近60日，V3因子 vs 参考因子）:')
print(corr_df.loc[['VolSpike', 'PriceVarShrink', 'PriceMeanShrink', 'VSSP (composite)'],
                   ['Reversal_5d', 'LowVol_20d', 'Momentum_20d', 'VWAP_Bias_10d', 'Amihud_20d', 'RSI_14d']].round(4).to_string())

In [ ]:
# ── 可视化：相关矩阵热力图 ───────────────────────────────────
fig, ax = plt.subplots(figsize=(12, 10), facecolor='#1A1A2E')
ax.set_facecolor('#16213E')

mask = np.isnan(avg_corr)
arr_vis = np.where(mask, 0, avg_corr)
im = ax.imshow(arr_vis, cmap='RdYlGn', vmin=-1, vmax=1, aspect='auto')

ax.set_xticks(range(n))
ax.set_yticks(range(n))
short_names = [nm.replace(' (composite)', '\n(composite)') for nm in names]
ax.set_xticklabels(short_names, color='#ECF0F1', fontsize=8, rotation=45, ha='right')
ax.set_yticklabels(short_names, color='#ECF0F1', fontsize=8)

for i in range(n):
    for j in range(n):
        txt_color = 'black' if abs(arr_vis[i, j]) < 0.7 else 'white'
        ax.text(j, i, f'{arr_vis[i,j]:.2f}', ha='center', va='center',
                 color=txt_color, fontsize=7.5)

plt.colorbar(im, ax=ax, fraction=0.03, label='截面平均 Pearson 相关')

# 在 V3 因子区域画矩形框
import matplotlib.patches as mpatches
rect = mpatches.FancyBboxPatch((-0.5, -0.5), 4, 4, linewidth=2,
                                 edgecolor='#FFD700', facecolor='none',
                                 boxstyle='square,pad=0')
ax.add_patch(rect)
ax.set_title('因子相关矩阵（近60日截面均值）\n黄框：V3 新因子内部相关', color='#ECF0F1', fontsize=11)

plt.tight_layout()
plt.show()
print('✅ 因子独立性分析完成')

## §10. 多因子融合（StatisticalCombiner）

将 V3 新因子与经典因子融合，验证是否提升组合 Sharpe

In [ ]:
from quant_alpha_engine.fusion import Labeler

# ── 前向标签 ──────────────────────────────────────────────────
y5 = Labeler.from_price(close, horizon=5)

# ── 方案一：纯参考因子（基准）───────────────────────────────
base_factors = [
    op.Rank(-op.Ts_Delta(close, 5)),             # 短期反转
    op.Rank(-op.Ts_Std(close.pct_change(), 20)),  # 低波动
    op.Rank(op.VWAP_Bias(close, volume, 10)),     # VWAP 乖离
]

# ── 方案二：纯参考因子 + V3 复合因子 ──────────────────────────
enhanced_factors = base_factors + [
    op.Rank(f_vssp),   # V3 复合因子
]

# ── 方案三：纯 V3 三子因子 ────────────────────────────────────
v3_factors = [
    op.Rank(f_volspike),
    op.Rank(-f_pvshrink),
    op.Rank(-f_pmshrink),
]

# ── 训练三种融合器 ────────────────────────────────────────────
print('训练 StatisticalCombiner (ic_weighted)...')
results_fusion = {}
for plan_name, factors_list in [
    ('基准（3参考因子）', base_factors),
    ('增强（+VSSP复合）', enhanced_factors),
    ('V3三子因子',        v3_factors),
]:
    combiner = StatisticalCombiner(method='ic_weighted')
    combiner.fit(factors_list, label=y5)
    res = combiner.evaluate(
        factors_list, close, is_susp, is_limit,
        rebalance_freq=REBALANCE_FREQ, top_n=TOP_N,
        weight_method=WEIGHT_METHOD, cost_rate=COST_RATE,
        delay=DELAY, decay=DECAY, industry=INDUSTRY,
        start_date=START_DATE, end_date=END_DATE,
    )
    results_fusion[plan_name] = res
    print(f'  ✅ {plan_name:<20}  '
          f'Sharpe={res.metrics["Sharpe_Ratio"]:+.4f}  '
          f'IC={res.metrics["IC_Mean"]:+.4f}  '
          f'MaxDD={res.metrics["最大回撤"]*100:+.2f}%  '
          f'权重={np.round(combiner.weights_,3)}')

In [ ]:
# ── 融合方案汇总对比 ──────────────────────────────────────────
key_cols = ['年化收益率', 'Sharpe_Ratio', '最大回撤', 'IC_Mean', 'ICIR', 'IC_胜率', 'Fitness']
rows_f = {name: {k: res.metrics.get(k, np.nan) for k in key_cols}
          for name, res in results_fusion.items()}
df_f = pd.DataFrame(rows_f).T

df_f_fmt = df_f.copy()
for c in ['年化收益率', '最大回撤', 'IC_胜率']:
    df_f_fmt[c] = df_f[c].apply(lambda v: f'{v*100:+.2f}%' if pd.notna(v) else 'N/A')
for c in ['Sharpe_Ratio', 'IC_Mean', 'ICIR', 'Fitness']:
    df_f_fmt[c] = df_f[c].apply(lambda v: f'{v:+.4f}' if pd.notna(v) else 'N/A')

print('\n======= StatisticalCombiner 多因子融合方案对比 =======')
print(df_f_fmt.to_string())

# ── 净值曲线对比 ─────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(14, 5), facecolor='#1A1A2E')
ax.set_facecolor('#16213E')
colors_f = ['#95A5A6', '#2ECC71', '#FF6B35']
lws_f    = [1.2, 2.0, 2.0]
for (name, res), color, lw in zip(results_fusion.items(), colors_f, lws_f):
    nav = res.nav.dropna()
    ax.plot(nav.index, nav.values, label=name, color=color, linewidth=lw)
ax.axhline(1.0, color='white', linewidth=0.5, alpha=0.3, linestyle='--')
ax.set_title('多因子融合方案净值对比（StatisticalCombiner ic_weighted）',
              color='#ECF0F1', fontsize=11)
ax.set_xlabel('日期', color='#ECF0F1', fontsize=9)
ax.set_ylabel('净值', color='#ECF0F1', fontsize=9)
ax.legend(fontsize=9, facecolor='#1A1A2E', edgecolor='#0F3460', labelcolor='#ECF0F1')
ax.tick_params(colors='#ECF0F1', labelsize=7)
ax.grid(True, color='#2F3640', alpha=0.5, linewidth=0.5)
for sp in ax.spines.values(): sp.set_color('#0F3460')
plt.tight_layout()
plt.show()
print('✅ 多因子融合分析完成')

## §11. 总结与参数建议

### V3 新增因子速查表

```python
from quant_alpha_engine.ops import AlphaOps as op

# ── 缩量稳价因子 (VolSpike-PriceStable) ──────────────────────

# 子因子一：换手率放大比（M1/M2）
# n_multiple = vol_mult（判定显著放量阈值，如 >2.0 倍）
f_vs = op.VolSpike(
    turnover,
    n_short    = 3,     # 近期窗口（天）
    n_long     = 10,    # 历史基准窗口（天）
    n_multiple = 2.0,   # vol_mult：M1 > M2×2.0 时视为显著放量
)

# 子因子二：价格方差收缩比（V1/V2）
# n_multiple = pricevar_shrink（收缩阈值，如 <0.5 倍）
f_pvs = op.PriceVarShrink(
    close,
    open_,              # 注意参数名带下划线（Python 关键字保护）
    n_short    = 3,
    n_long     = 10,
    n_multiple = 0.5,   # pricevar_shrink：V1 < V2×0.5 时视为显著收缩
)

# 子因子三：均价下移比（P1/P2）
# n_multiple = price_shrink（下移阈值，如 <0.98 倍）
f_pms = op.PriceMeanShrink(
    close,
    n_short    = 3,
    n_long     = 10,
    n_multiple = 0.98,  # price_shrink：P1 < P2×0.98 时视为显著下移
)

# 复合因子：三重信号加权（ZScore 标准化后等权）
f_vssp = op.VolSpikeStablePrice(
    turnover,
    close,
    open_,
    n_short         = 3,    # 统一短期窗口
    n_long          = 10,   # 统一长期窗口
    n_multiple      = 2.0,  # 基础倍数参数（可被下方单独参数覆盖）
    vol_mult        = 2.0,  # 覆盖 VolSpike 的阈值
    pricevar_shrink = 0.5,  # 覆盖 PriceVarShrink 的阈值
    price_shrink    = 0.98, # 覆盖 PriceMeanShrink 的阈值
)

# 策略使用：只保留三重信号同时满足的股票
triple_mask = (
    (f_vs  > 2.0)  &   # 换手放大 2 倍以上
    (f_pvs < 0.5)  &   # 方差收缩到历史的 50% 以下
    (f_pms < 0.98)     # 均价小幅下移（≥2%）
)
filtered_factor = f_vssp.where(triple_mask)  # 不满足条件的设为 NaN
```

---

### 参数推荐

| 参数 | 推荐值 | 调整方向 |
|------|--------|----------|
| `n_short` | 3 天 | 更短（1-2天）= 更敏感；更长（5-7天）= 更稳定 |
| `n_long` | 10 天 | 与 n_short 保持 3-5 倍关系 |
| `vol_mult` | 2.0 | 市场活跃时可提高至 2.5-3.0 |
| `pricevar_shrink` | 0.5 | 越小 = 信号越稀少越纯粹 |
| `price_shrink` | 0.98 | 0.95-0.99 之间调节 |

### 适用场景

| 场景 | 建议 |
|------|------|
| 单独使用 | 直接用 `op.Rank(f_vssp)` 作为选股因子 |
| 条件筛选 | 结合 `triple_mask` 过滤后再排名 |
| 因子融合 | 与低相关的参考因子（如 Reversal、LowVol）组合，提升 Sharpe |
| 实盘数据 | 建议将 `turnover` 替换为真实换手率（日成交量 / 流通股本）|

In [ ]:
# ── 最终 VolSpikeStablePrice 完整回测（最优参数）────────────────
best_factor = op.Rank(f_vssp)

result_final = VectorEngine(
    factor         = best_factor,
    close          = close,
    is_suspended   = is_susp,
    is_limit       = is_limit,
    rebalance_freq = REBALANCE_FREQ,
    top_n          = TOP_N,
    weight_method  = WEIGHT_METHOD,
    cost_rate      = COST_RATE,
    delay          = DELAY,
    decay          = DECAY,
    industry       = INDUSTRY,
    start_date     = START_DATE,
    end_date       = END_DATE,
).run()

print('=' * 55)
print('  VolSpikeStablePrice 完整回测结果')
print('=' * 55)
result_final.print_summary()
result_final.plot()